In [1]:
import json
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
import joblib

In [4]:
df=pd.read_csv('Stockprice.csv')
##df.drop(['<OpenFixed>','<HighFixed>','<LowFixed>','<CloseFixed>','<VolumeDeal>','<VolumeFB>','<VolumeFS>'],1,inplace = True)
##df[['date']] = df[['<DTYYYYMMDD>']].applymap(str).applymap(lambda s: "{}/{}/{}".format(s[4:6],s[6:], s[0:4]))
##df.drop(['<DTYYYYMMDD>'],1, inplace = True)
df

,Ticker,DTYYYYMMDD,Volume,Open,High,Low,Close,date,Year,Doy
0,AAA,20201104,996980,11.05000,11.30000,11.00000,11.20000,4 Nov 2020,2020,309
1,AAA,20201103,556250,11.05000,11.05000,10.95000,11.00000,3 Nov 2020,2020,308
2,AAA,20201102,634490,10.95000,11.00000,10.85000,11.00000,2 Nov 2020,2020,307
3,AAA,20201030,1028570,11.00000,11.00000,10.80000,10.90000,30 Oct 2020,2020,304
4,AAA,20201029,1207870,11.00000,11.05000,10.80000,10.85000,29 Oct 2020,2020,303
...,...,...,...,...,...,...,...,...,...,...
1439760,YEG,20180702,10610,296.70001,296.70001,296.70001,296.70001,2 Jul 2018,2018,183
1439761,YEG,20180629,1800,321.00000,321.00000,319.00000,319.00000,29 Jun 2018,2018,180
1439762,YEG,20180628,76700,342.00000,343.39999,342.00000,343.00000,28 Jun 2018,2018,179
1439763,YEG,20180627,16860,321.00000,321.00000,321.00000,321.00000,27 Jun 2018,2018,178


In [8]:
df['7 Days STD']=df.groupby('Ticker').rolling(window=7)['Close'].std().reset_index(drop=True)
df['7 Days MA']=df.groupby('Ticker').rolling(window=7)['Close'].mean().reset_index(drop=True)
df['14 Days MA']=df.groupby('Ticker').rolling(window=14)['Close'].mean().reset_index(drop=True)
df['21 Days MA']=df.groupby('Ticker').rolling(window=21)['Close'].mean().reset_index(drop=True)
df['H-L']=df['High']-df['Low']
df['C-O']=df['Close']-df['Open']
df.replace([np.inf, -np.inf],np.nan, inplace = True)
df.dropna(inplace=True)
df

,Ticker,DTYYYYMMDD,Volume,Open,High,Low,Close,date,Year,Doy,7 Days STD,7 Days MA,14 Days MA,21 Days MA,H-L,C-O
20,AAA,20201007,1682820,12.50000,12.55000,12.35000,12.35000,7 Oct 2020,2020,281,0.069864,12.221429,12.014286,11.676190,0.20000,-0.15
21,AAA,20201006,3310270,12.55000,12.75000,12.45000,12.55000,6 Oct 2020,2020,280,0.141000,12.271429,12.085714,11.740476,0.30000,0.00
22,AAA,20201005,4522700,12.25000,12.65000,12.15000,12.45000,5 Oct 2020,2020,279,0.155456,12.300000,12.142857,11.809524,0.50000,0.20
23,AAA,20201002,2669380,12.25000,12.35000,11.95000,12.15000,2 Oct 2020,2020,276,0.155456,12.300000,12.175000,11.864286,0.40000,-0.10
24,AAA,20201001,1474770,12.30000,12.30000,12.20000,12.25000,1 Oct 2020,2020,275,0.143510,12.314286,12.214286,11.928571,0.10000,-0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439760,YEG,20180702,10610,296.70001,296.70001,296.70001,296.70001,2 Jul 2018,2018,183,24.073739,256.085717,250.414285,242.623808,0.00000,0.00
1439761,YEG,20180629,1800,321.00000,321.00000,319.00000,319.00000,29 Jun 2018,2018,180,33.855746,265.014289,255.707143,248.176190,2.00000,-2.00
1439762,YEG,20180628,76700,342.00000,343.39999,342.00000,343.00000,28 Jun 2018,2018,179,42.569187,279.628574,262.714286,254.147618,1.39999,1.00
1439763,YEG,20180627,16860,321.00000,321.00000,321.00000,321.00000,27 Jun 2018,2018,178,37.156328,293.342860,268.571429,258.295237,0.00000,0.00


In [25]:
X = df[['<Volume>','21 Days MA','14 Days MA','7 Days MA','7 Days STD']]
Y = df['<Close>']
X

,<Volume>,21 Days MA,14 Days MA,7 Days MA,7 Days STD
20,1682820,11.676190,12.014286,12.221429,0.069864
21,3310270,11.740476,12.085714,12.271429,0.141000
22,4522700,11.809524,12.142857,12.300000,0.155456
23,2669380,11.864286,12.175000,12.300000,0.155456
24,1474770,11.928571,12.214286,12.314286,0.143510
...,...,...,...,...,...
2563,75800,46.585714,47.421429,45.914286,1.436928
2564,328700,46.619048,47.292857,45.371429,0.468025
2565,104900,46.576190,47.100000,45.557143,0.547288
2566,109200,46.566667,46.992857,45.614286,0.628301


In [26]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = .20)

In [27]:
lr_model=LinearRegression()
lr_model.fit(x_train,y_train)
y_pred=lr_model.predict(x_test)

In [28]:
kfold = model_selection.KFold(n_splits=20, random_state =100)
result_kfold = model_selection.cross_val_score(lr_model,x_test,y_test,cv=kfold)
print("Accuracy: ", result_kfold.mean()*100)

Accuracy:  98.95683763740047


C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [5]:
p_stock.dump(lr_model,"./linear_regression.joblib",compress=True)

{'<Volume>': 80400.0, '21 Days MA': 12.257142857142853, '14 Days MA': 12.54999999999998, '7 Days MA': 12.971428571428643, '7 Days STD': 0.048795003649805345}
